This file was created to scrap the ERCOT load data from: https://www.ercot.com/gridinfo/load/load_hist/

Long story short, it is stored in multiple file types and there since there are only 20(ish) that we are interested in, it was simpler to just grab the files individually.

In [1]:
#Basics
import numpy as np
import pandas as pd

import os
import requests
import time
from bs4 import BeautifulSoup as BSoup

1) Get the request/beautifulsoup of the main page: https://www.ercot.com/gridinfo/load/load_hist/

2) From that page, pull all the excel hrefs into a list

3) Loop over that list and for each URL, read the excel document into pandas

4) For the next iteration of the loop read the info and concat the dataframes together (keep growing the dataframe)

5) Save the resulting dataframe as a CSV so we don't have to run the scrapping again

6) Save a shorter dataframe (with just the coastal data) as a CSV

7) Drop all this acquire into a function


In [2]:
headers = {'User-Agent': 'Codeup Data Science'}

In [3]:
response = requests.get('https://www.ercot.com/gridinfo/load/load_hist/',headers=headers)
response.status_code

200

In [4]:
soup = BSoup(response.text,'html.parser')

**Page Inspection Notes:**

href examples:

href="https://www.ercot.com/files/docs/2022/02/08/Native_Load_2022.zip" title="2022 ERCOT Hourly Load Data " download="" class="d-block pb-2">2022 ERCOT Hourly Load Data 

href="https://www.ercot.com/files/docs/2021/11/12/Native_Load_2021.zip" title="2021 ERCOT Hourly Load Data" download="" class="d-block pb-2">2021 ERCOT Hourly Load Data

In [5]:
len(soup.select('a',{'class':'d-block pb-2'}))

263

In [6]:
soup.select('a',{'class':'d-block pb-2'})


[<a href="#" onclick="navigate(pageProxySecureContextRequest);">MIS LOG IN</a>,
 <a href="https://www.ercot.com" id="hlogo">
 <svg aria-label="Electric Reliability Council of Texas (ERCOT)" role="img" viewbox="0 0 226.91 83.27" xmlns="http://www.w3.org/2000/svg">
 <defs>
 <style>
             .erc-1 {
               fill: #5d6770;
             }
 
             .erc-2 {
               fill: #00abc7;
             }
           </style>
 </defs>
 <title>ERCOT-logo</title>
 <path class="erc-1" d="M29.43,49.05c-2.89,9.26-8.73,13.56-17.94,13.56-4.58,0-7.81-1.15-9.62-3.43C-.18,56.59-.55,52.42.78,46.77L3.93,33.3C6.49,22.34,12.6,17,22.61,17c4.58,0,7.81,1.15,9.62,3.43,2.05,2.59,2.42,6.76,1.1,12.41l-1.56,6.86a1.12,1.12,0,0,1-1.21.91H11.34c-.76,0-1.07,0-1.24.77L8.55,48c-.76,3.26-.57,5.56.59,7a5,5,0,0,0,4.18,1.68c4.14,0,6.9-2.67,8.21-7.94a.8.8,0,0,1,.75-.58h6.55a.57.57,0,0,1,.6.85M20.78,22.9c-4.22,0-7,2.77-8.28,8.25l-.44,2.05a1.7,1.7,0,0,0,0,1.07c.2.46.52.46,1.16.46H23.82c.77,0,1-.1,1.19-.83l.54-2.3

In [7]:
#<div class="col-12 col-md">
len(soup.select('div',{'class':'row'}))

235

In [8]:
len(soup.select('div',{'id':'details-70372353-7ccc-4458-b9d8-ebb5841796c0'}))

235

In [9]:
len(soup.select('div',{'id':'bcrumb'}))

235

In [10]:
len(soup.find_all('a', href=True, title=True))

29

In [11]:
for a in soup.find_all('a', href=True, title=True):
    print(a['href'])

https://www.ercot.com/files/docs/2022/02/08/Native_Load_2022.zip
https://www.ercot.com/files/docs/2021/11/12/Native_Load_2021.zip
https://www.ercot.com/files/docs/2021/01/12/Native_Load_2020.zip
https://www.ercot.com/files/docs/2020/01/09/Native_Load_2019.zip
https://www.ercot.com/files/docs/2019/01/07/native_load_2018.zip
https://www.ercot.com/files/docs/2018/01/09/native_load_2017.zip
https://www.ercot.com/files/docs/2017/01/10/native_Load_2016.zip
https://www.ercot.com/files/docs/2016/01/07/native_load_2015.xls
https://www.ercot.com/files/docs/2015/10/22/2014_ercot_hourly_load_data.xls
https://www.ercot.com/files/docs/2015/10/22/2013_ercot_hourly_load_data.xls
https://www.ercot.com/files/docs/2015/10/22/2012_ercot_hourly_load_data.xls
https://www.ercot.com/files/docs/2015/10/22/2011_ercot_hourly_load_data.xls
https://www.ercot.com/files/docs/2015/10/22/2010_ercot_hourly_load_data.xls
https://www.ercot.com/files/docs/2015/10/22/2009_ercot_hourly_load_data.xls
https://www.ercot.com/fi